# Ciencia de datos
## Práctica 4. Visualización de la información con plotly
### Alberto Benavides

De los datos correspondientes a los registros obtenidos de los PDFs de la Secretaría de Salud de México

In [2]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

data1 = pd.read_csv("D:/FIME/Epidemia/Data/csvSemanales/nacional.csv")
data1.loc[:, 'cie'] = data1['cie'].astype(str).str[0]

# https://plot.ly/python/table/
trace = go.Table(
    header=dict(
        values=list(data1),
        fill = dict(color='#a1c3d1'),
    ),
    cells=dict(values=data1.sample(10).T)) # https://stackoverflow.com/a/19483025

table = [trace] 
py.iplot(table, filename = 'semanalesTodas')

C:\Users\bena8\AppData\Local\Programs\Python\Python37-32\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



y estos mismos preprocesados

In [3]:
data2 = pd.read_csv("D:/FIME/Epidemia/Data/semanalesTodasKmeans.csv")
trace = go.Table(
    header=dict(
        values=list(data2),
        fill = dict(color='#a1c3d1'),
    ),
    cells=dict(values=data2.sample(10).T)) # https://stackoverflow.com/a/19483025

table = [trace] 
py.iplot(table, filename = 'semanalesTodasKmeans')

se pueden mostrar visualizaciones de los resultados de las estadísticas básicas reportadas. Empezaremos por mostrar un diagrama de cantidad de registros por letra inicia de CIE de los datos extraidos de los PDFs

In [4]:
 # https://plot.ly/python/pie-charts/
x = data1['cie'].value_counts()

values = x.values

x = x.to_frame().T

labels = x.columns.values

trace = go.Pie(labels = labels, values = values)

py.iplot([trace], filename='conteoCIE')

De modo que las enfermedades cuya CIE inicia con la letra A ocupan el 42.2% de los registros. 

Ahora bien, se puede obtener una descripción de los datos agrupados por número de casos registrados y letra inicial de CIE

In [5]:
# https://plot.ly/python/box-plots/
boxes = []
for cie in data1.groupby(['cie']):
    trace = go.Box(y=cie[1]['casos'], name=str(cie[0]))
    boxes.append(trace)
py.iplot(boxes, filename='boxplotCIE')

Quizás una mejor visualización del resto de enfermedades, se podría obtener al remover las que inician con j

In [6]:
boxes = []
for cie in data1.groupby(['cie']):
    if cie[0] != 'j':
        trace = go.Box(y=cie[1]['casos'], name=str(cie[0]))
        boxes.append(trace)
py.iplot(boxes, filename='boxplotCIE-j')

Al preprocesar los datos, se agrupan las enfermedades por primera letra de la CIE y se descubre que los grupos A y B contienen la mayoría de los registros, contando un 31.19% y un 19.57% respectivamente.

In [7]:
# https://stackoverflow.com/a/51453257
pd.options.display.max_columns
data = data2

x = data['cie'].value_counts()

values = x.values

x = x.to_frame().T

labels = x.columns.values

trace = go.Pie(labels = labels, values = values)
py.iplot([trace], filename='conteoCIECluster')

In [8]:
boxes = []
for cie in data.groupby(['cluster']):
    if cie[0] != 'j':
        trace = go.Box(y=cie[1]['# m'], name=str(cie[0]))
        boxes.append(trace)
py.iplot(boxes, filename='clusterM')

KeyError: '# m'

In [9]:
boxes = []
for cie in data.groupby(['cluster']):
    if cie[0] != 'j':
        trace = go.Box(y=cie[1]['f1'], name=str(cie[0]))
        boxes.append(trace)
py.iplot(boxes, filename='clusterf1')

In [10]:
import matplotlib as mpl 
mpl.use('agg')
import matplotlib.pyplot as plt 
import numpy as np

for cie in data.groupby(['cluster']):
    fig = plt.figure(1)
    ax = fig.add_subplot(111)
    boxes = None
    boxes = []
    for feature in cie[1]:
        if feature == "enf":
            break
        else:
            # https://stackoverflow.com/a/28396902
            boxes.append(np.array(cie[1][feature]).astype(np.float))
    bp = ax.boxplot(boxes)
    plt.xticks(range(1, 12), cie[1])
    fig.savefig('{}.png'.format(cie[0]), bbox_inches='tight')
    plt.close(fig)

16
8
19
21
6
30
28


In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
ti = data.loc[:, data.columns != 'enf']

ti = ti.loc[:, ti.columns != 'cie']
ti = ti.loc[:, ti.columns != 'cluster']
features = pca.fit_transform(ti)

plt.close(fig)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features[:,0], features[:,1], features[:,2], c=data['cluster'])
plt.savefig("scater3d.png")
plt.close(fig)

In [12]:
pca = PCA(n_components=2)

features = pca.fit_transform(ti)

fig = plt.figure()
plt.scatter(features[:,0], features[:,1], c=data['cluster'])
plt.savefig("scater.png")
plt.close(fig)

In [14]:
import matplotlib as mpl 
mpl.use('agg')
import matplotlib.pyplot as plt 
import numpy as np

plt.figure()

for cie in data.groupby(['cluster']):
    boxes = []
    for feature in cie[1]:
        if feature != "enf":
            # https://stackoverflow.com/a/28396902
            boxes.append( np.array(cie[1][feature]).astype(np.float))
    plt.boxplot(boxes, positions = [1 + cie[0] * 13, 2 + cie[0] * 13, 3 + cie[0] * 13, 4 + cie[0] * 13, 5 + cie[0] * 13, 6 + cie[0] * 13, 7 + cie[0] * 13, 8 + cie[0] * 13, 9 + cie[0] * 13, 10 + cie[0] * 13, 11 + cie[0] * 13, 12 + cie[0] * 13, 13 + cie[0] * 13])
    if cie[0] > 2:
        break
plt.xlim(0, 40)
plt.savefig('todas.png', bbox_inches='tight')

ValueError: could not convert string to float: 'A'

También se obtuvieron las [correlaciones](https://www.dummies.com/education/math/statistics/how-to-interpret-a-correlation-coefficient-r/) existentes entre las características de los datos preprocesados, siendo de interés aquéllas que guardan correlación con el tipo de cluster asignado por $k$-medias

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html
data['cie'], uniques = pd.factorize(data['cie']) 

#https://stackoverflow.com/a/19483025
#print(list(data.corr()))

trace = go.Heatmap(z=data.corr().values, x = list(data.corr()), y= list(data.corr()))
corr=[trace]
py.iplot(corr, filename='corr')

Finalmente, se seleccionan las características de los datos

In [ ]:
features = ["m", "f1", "f2", "f3", "f4", "ac1", "ac2", "ac4", "ac8", "ac16", "ac32"]
x = data.loc[:, features].values

se normalizan 

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
from sklearn.preprocessing import MinMaxScaler
x = MinMaxScaler().fit(x).transform(x)

y con estas características normalizadas se puede hacer una selección a partir del umbral de varianza

In [ ]:
# https://stackoverflow.com/a/7670325
print("Columnas iniciales = {}".format(x.shape[1]))

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html#sklearn.feature_selection.VarianceThreshold
from sklearn.feature_selection import VarianceThreshold
th = 0.05 # .8 * (1 - .8)
print("Umbral de varianza = {}".format(th))
sel = VarianceThreshold(threshold=th)
x = sel.fit_transform(x)
# https://stackoverflow.com/q/39812885
dataSelected = data[data.columns[sel.get_support(indices=True)]]

y mostrar sus correlaciones

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
#print(data[['cie']])
dataSelected = dataSelected.assign(cluster=data[['cie']])

trace = go.Heatmap(z=dataSelected.corr().values, x = list(dataSelected.corr()), y= list(dataSelected.corr()))
corr=[trace]
py.iplot(corr, filename='corrSelected')

In [ ]:
trace = go.Scatter(
    x = data['ac2'],
    y = data['f1'],
    mode = 'markers',
     marker=dict(
        color=labels.astype(data['cluster'])
    )
)

scatter = [trace]

py.iplot(scatter, filename='basic-scatter')